##### Creation Date:
##### February 19 2022
##### Created By Alperen KOLAMUC

In [1]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics

In [2]:
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_feather("tmp/bulldozers_1")

#### Missing Value

* Bir `DataFrame`'de verimde sadece bir sütununda ve ya bir kaç sutununda `NA` yani boş veri olabilir. Bu boş verileri nasıl temizleriz? komple o satırı silsem; e o satırdaki belki çok önemli diğer bilgileride silmiş olacağım. Bu bir sorun. Peki o zaman o sütuna bakar en sık görülen değeri oraya yazabilirim? Evet bu bir çözüm olurdu ancak sırf bir veri çok görülüyor diye onu yazmak tam anlamıyla çözüm olmaz. Mesela bir taksi şirketinde 3 farklı renkte taksi olsun sarı, mavi, gri. Sarı renkten 100 tane, mavi'den 50 tane ve gri'den 10 tane olsa ve fiyat ve komfor olarak gri>mavi>sarı olsa şimdi gri takside renk değeri girilmemiş ama fiyat ve komfor değerleri girilmiş sırf çok var diye sarı renk yazarsam bu sefer model sarı renkli arabaların komfor ve fiyat değerlerine + yönde bir ekleme yapacak. Bu sorunun en makul çözümlerinden biri o sütunun ortalamasını alırım ve ortalama değeri boş kutuya yazarım. Model gelip bu değeri alır ama burada bir eksik daha var. Bu değerin gerçek bir değer olup olmadığını model bilmiyor. Bu değerin bizim oluşturduğumuz tahmin bir değer olduğunuda belirtmemiz lazım. O zaman `NA` olan bir sutunun yanına birde <`NA'lı sutun ismi` + `_fill_NA`> adında bir sutun daha koyarım. Eğer o satıdaki değeri ben yazmışsam `_fill_NA` sutnundaki değeri `True` yaparım eğer ben yazmamışsam `_fill_NA` sutunundaki değeri `False` yaparım. Böylece model o kutudaki değerin gerçek değer olup olmadığınında farkına varır

|   | Renk | Fiyat | Komfor (%) |
|:-:|:----:|:-----:|:----------:|
| 1 | Sarı |  10K  |     40     |
| 2 | Sarı |  10K  |     40     |
| 3 | Mavi |  20K  |     60     |
| 4 |  NA  |  50K  |     95     |
| 5 | Sarı |  10K  |     40     |

|   | Renk | Renk_Fill_Na | Fiyat | Komfor (%) |
|:-:|:----:|:------------:|:-----:|:----------:|
| 1 | Sarı |     False    |  10K  |     40     |
| 2 | Sarı |     False    |  10K  |     40     |
| 3 | Mavi |     False    |  20K  |     60     |
| 4 | Sarı |     True     |  50K  |     95     |
| 5 | Sarı |     False    |  10K  |     40     |

In [5]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum(): # if pd.isnull(col).sum != 0, col.isnull().sum() != 0
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())

In [6]:
# Print has null columns
for n,c in df.items():
    if is_numeric_dtype(c):
        if df[n].isnull().sum():
            print(n)

auctioneerID
MachineHoursCurrentMeter


In [7]:
# Print Nan Values sum
df["MachineHoursCurrentMeter"].isnull().sum()

249575

In [8]:
# Fixing NaN Values
fix_missing(df, df["MachineHoursCurrentMeter"],"MachineHoursCurrentMeter")

In [9]:
df["MachineHoursCurrentMeter"].isnull().sum()

0

In [10]:
df["MachineHoursCurrentMeter_na"]

0          True
1          True
2          True
3          True
4          True
          ...  
339995    False
339996    False
339997    False
339998    False
339999    False
Name: MachineHoursCurrentMeter_na, Length: 340000, dtype: bool